In [ ]:
###################################################
# Profile Mask Visualizer
###################################################

from cgmbrush.cgmbrush import *
import matplotlib.colors as colors
from cgmbrush.cosmology import cosmology as cosmo
from cgmbrush.cosmology import halo as halo
%matplotlib inline


In [ ]:
def view_mask(profile: CGMProfile, mass):

    provider = BolshoiProvider()

    # pick some parameters
    redshift = 0 
    resolution = 16
    cellsize = provider.Lbox / (provider.halofieldresolution * resolution)
    print("cellsize (kpc): %s" % (cellsize * 1000))
    fine_mask_len = 20 * resolution
    fine_mask_len_small = 2 * resolution

    comoving_rvir = halo.comoving_rvir(cosmo, mass, redshift)
    print("co-moving virial radius (kpc): %s" % (comoving_rvir * 1000))

    mask = profile.get_mask(mass, comoving_rvir, redshift, resolution, 1, cellsize, fine_mask_len)
    mask_small = profile.get_mask(mass, comoving_rvir, redshift, resolution, 1, cellsize, fine_mask_len_small)
    fig, ax = plt.subplots(1,2,figsize=(24, 12))
    pos = ax[0].imshow(mask)
    pos = ax[1].imshow(mask_small)
    ax[0].title.set_text('{} Profile, M=${:.1e}$'.format(profile.name, mass))
    ax[1].title.set_text('{} Profile (zoomed), M=${:.1e}$'.format(profile.name, mass))
    fig.colorbar(pos, ax=ax)

# Choose a profile and mass
view_mask(FireProfile(), 1E10) 
view_mask(FireProfile(), 1E14) 
view_mask(NFWProfile(), 1E10) 
view_mask(NFWProfile(), 1E14) 

In [ ]:
from cgmbrush import *
def compareRadii(mass, z):
    print("Mass {:.1e}, z={}".format(mass, z))
    a = halo.comoving_rvir(cosmo, mass, z)
    b = halo.r200Mz(cosmo, mass, z)
    print('  rVir:\t{:.4f} Mpc'.format(a))
    print('  r200:\t{:.4f} Mpc'.format(b))
    print('  difference:\t{:.1f}%'.format((abs(b-a)) * 100 / b))

masses = [1E10,1E13,1E15]
redshifts = [0,0.5,1,2]

for z in redshifts:
    for m in masses:
        compareRadii(m, z)

In [ ]:
provider = BolshoiProvider()
halos = provider.get_halos(0)
print("Max halo size: {:.1e}".format(max(halos['Mvir'])))